In [43]:
!python --version

Python 3.6.12 :: Anaconda, Inc.


In [84]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [ ]:
# 워닝 없애주는 것
import warnings  
warnings.filterwarnings('ignore')

In [85]:

# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm

In [46]:
# query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
# # query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# # query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: '|)

# start_date = input('4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
# end_date = input('5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')

In [86]:
query_txt='넷플'
start_date="20200601"
end_date="20200731"

In [87]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 '이재용'
element.submit()

#Step 3. "블로그" 카테고리 선택
driver.find_element_by_link_text("블로그").click( )    # .click() 괄호 안을 눌러라는 뜻

#Step 4. 오른쪽의 검색 옵션 버튼 클릭
driver.find_element_by_id("_search_option_btn").click( )

#Step 5. 정렬 : "관련도순" 
# 개발자 도구에서 정렬 버튼의 id 가 보이지 않습니다.
# 이럴 경우 쉽게 사용할 수 있는 방법이 바로 xpath 를 이용하는 방법입니다.
# xpath는 개발자 도구에서 해당 메뉴 부분을 마우스 오른쪽 버튼을 누르고 
# copy -> copy xpath 를 선택하면 됩니다
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click( ) # 관련도순 xpath

#Step 6. 날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

# 이 부분이 아주 중요합니다.
# 날짜 부분에 날짜를 입력할 때 입력 속도가 너무 빠를 경우 날짜가 입력이 되다가 
# 오타가 나오는 경우가 많습니다.
# 그래서 날짜를 입력할 때 for 반복문을 사용해서 1 글자씩 입력하도록 코딩했습니다.

# 시작 날짜 입력하기
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)
    
# 종료 날짜 입력하기
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)
    
# Step7. 날짜 입력 "적용하기" 버튼을 클릭 합니다.  
driver.find_element_by_class_name("tx").click()
time.sleep(3)

# # Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
# ele2 = driver.find_element_by_id("inpop3")
# ele2.send_keys(query_txt2)
# ele2.send_keys(',')
# ele2.send_keys(query_txt3)
# driver.find_element_by_css_selector(".btn_ft.ty_green._search").click( )

## 글 url 크롤링 시작

In [88]:
url_list = []
title_list = []

# ★몇개의 페이지를 크롤링할지 선택
total_page = 100
for i in tqdm(range(0, total_page)):  # 페이지 번호
    i = i*10 + 1
    url = "https://search.naver.com/search.naver\
?date_from={0}&date_option=8&date_to={1}\
&dup_remove=1&nso=p%3Afrom{2}to{3}post_blogurl=\
&post_blogurl_without=&query={4}&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date,query_txt, i)
    driver.get(url)
    time.sleep(2)
    
    # URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    article_raw = driver.find_elements_by_css_selector(titles)
#     article_raw

    # url 크롤링 시작    
    for article in article_raw:
        url = article.get_attribute('href')   
        url_list.append(url)
    
    # 제목 크롤링 시작    
    for article in article_raw:
        title = article.get_attribute('title')   
        
        
        
        title_list.append(title)
    
        print(title)
    
print('url갯수: ', len(url_list))
print('url갯수: ', len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
df.to_excel("C:/Users/user/Desktop/project/netflix_20200601_20200731.xlsx")

넷플릭스 다큐멘터리 1탄 ; 넷플 다큐 시리즈
넷플 2020 신작 리뷰 스핀아웃/빨간 머리 앤 시즌3/위쳐
메탈슬러그5 넷플(17년 전쟁의 최종 결전 직전 최종 플레이)
2019.9월~현재 넷플+웨이브 영화&드라마
[넷플] 강철의 연금술사: 브라더후드 - 내 최애 애니
넷플 시작!
이젠 넷플보다 왓챠플레이! 요금제와 동시접속파헤치기
넷플릭스 중국드라마 추천! 넷플 중드 입문자를 위하여
[hazelist] 멤버십 커뮤니티 후보 : 헤이조이스 / 코사이어티 / 넷플연가
넷플릭스 중드/대드 목록 정리: 중드 입문자 넷플 중국 드라마 추천 ②
송위룡 下一站是幸福 (하일참시행복:한국넷플제목: 누나의 첫사랑) 드뎌 방송! (방송기념 새 예고영상&움짤)
[1줄컷/1줄평] <한드> 넷플 '나 홀로 그대' <전편감상> : 사랑도 인공지능과..
넷플,!
<오티스의비밀상담소> 넷플추천 드라마
첫 넷플 주접
2020.2월 넷플 감상
메탈슬러그2 넷플
[넷플] 연희공략: 금지옥엽 延禧攻略: 金枝玉叶 최고야!!
왓챠 버리고 넷플릭스 고인물이 된 자의 넷플 my list 공개! (미드/하이틴/로맨스/한국영화/드라마추천)
넷플 오리지널 하이틴 띵작 [내가 사랑했던 모든 남자들에게] :
넷플릭스 :: 넷플 공무원 핫가이 노아 센티네오 덕질 + 출연작 추천
넷플릭스 추천 : 루머의 루머의 루머 + 넷플 동시자막 하는 법
Anne with an E 시즌3 넷플 있으면 꼭보세오ŏ̥̥̥̥םŏ̥̥̥̥
넷플쉐도잉 내가사랑했던모든남자들 & 헌혈은오늘도실패;;
[중드 내하보스요취아] 서개빙, 왕쌍 내하보스요취아. 넷플에서 중드가 보고싶다? '보스가 결혼하재요' 보세여!
[어쩌다 만보] 한남동 편 (현대카드 뮤직라이브러리/경리단길/메시야/넷플연가)
(어플)Cake_event ~Day4까지 + (넷플)빨간머리앤 시즌1_단어
넷플은 무조건 피씨나 티비로
키싱 부스: the KISSING BOOTH( 넷플)
넷플에 드디어..!!
[일상] 191230-200104 내 이화그린 야잠과 넷플
[넷플 추천] 넷플릭스 

In [ ]:
! pip install openpyxl

In [ ]:
! pip install IProgress

In [ ]:
! pip install ipywidgets

In [ ]:
conda install -c conda-forge ipywidgets

In [ ]:
conda install -c conda-forge nodejs

In [ ]:
conda install -c conda-forge ipywidgets

## 본문 크롤링 하기

In [82]:
import sys
import os
import pandas as pd
import numpy as np

In [ ]:
!pip install xlrd

In [83]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("netflix_20200601_20200731.xlsx")        # 기본 모델

num_list = len(url_load)

print(num_list)
url_load

1000


,Unnamed: 0,url,title
0,0,https://blog.naver.com/miroahn_44?Redirect=Log...,넷플 루시퍼
1,1,https://blog.naver.com/liemk?Redirect=Log&logN...,밤에 잠이 안올때 넷플 보기도 이제 한계
2,2,https://blog.naver.com/sjw6219?Redirect=Log&lo...,넷플 11화 사진+움짤
3,3,https://whitealmond.blog.me/222084820337,마음의 위안이 되어 줄 넷플 미드 추천
4,4,https://blog.naver.com/kiss8483?Redirect=Log&l...,드디어 한다. 넷플 추천(1)
...,...,...,...
995,995,https://blog.naver.com/kyrin1004?Redirect=Log&...,[뉴질랜드 워홀]+486 이사준비/Chemist Warehouse 비타민 온라인 구...
996,996,https://blog.naver.com/sts07024?Redirect=Log&l...,[넷플추천]더크 젠틀리의 전체론적 탐정 사무소
997,997,https://blog.naver.com/jmbaek1009?Redirect=Log...,워리어 넌 신의 뜻대로 대만 원작 만화를 바탕으로한 판타지 드라마(넷플X스 시청가능)
998,998,https://blog.naver.com/violeta3_3?Redirect=Log...,넷플지옥


In [53]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

# ★수집할 글 갯수
number = 500
for i in tqdm(range(0, number)): 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-"                                 
        tit = driver.find_element_by_css_selector(overlays)         # title
        title = tit.text
        title

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nick
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # date
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)         # date

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("netflix_20200601_20200731.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 저장하기
result_df.to_excel("netflix_20200601_20200731.xlsx")

1 
2 넷플릭스 365일 정상을 차지하다
3 넷플릭스 호러 스릴러 영화 추천 <데블>: 우연도 자비도 없는~ 
4 메시아 - 넷플릭스
5 [넷플릭스] 웹툰 작가 최규석 & 연상호  지옥(
6 넷플릭스 [도시괴담] - 빠르게, 부담 없이 즐길 수 있는 '짤방' 같은 영상
7 지난 주 크리스토퍼 놀란 감독이 의도치 않게 모두의 주목을 받았다. “놀란이 영화 촬영장에서 의자를 허용하지 않는다”는 앤 해서웨이의 말이 오해를 사자, 놀란 측이 “배우나 제작진은 앉아도 되지만, 감독 본인은 앉는 걸 선호하지 않는다”라며 해명한 것이다. <저스티스 리그>의 사이보그 레이 피셔는 “조스 웨던 감독이 폭력적이고 프로답지 못했으며, 이를 당시 워너브러더스와 DC 임원진이 용인했다”고 폭로했다. 하지만 <저스티스 리그> 제작자인 존 버그는 의혹을 강력히 부인하면서 오히려 피셔가 “프로답지 못했다”고 비판했다. 사실 여부는 당사자들만 알겠지만, 폭로전까지 벌어지는 걸 보니 잭 스나이더 감독판이 더 기다려진다. 그 외에 주목받거나 주목할 만한 말들을 정리했다.
9 넷플릭스 래치드 생각보다 지루하네요 중도 포기각
11 [사탄의 베이비시터2 킬러퀸] 전작보다 신박하진 않지만, 괜찮은 결말로 마무리 지은 넷플릭스 오리지널 영화 추천
13 
14 넷플릭스 도시괴담, 기생수 한바탕
15 도시괴담 
16 방콕하는 자들에게 카타르시스 선사할 넷플릭스 액션 영화 추천
17 넷플릭스에서 1화만 본 드라마들
18 세계적인 엔터테인먼트 스트리밍 서비스 넷플릭스(Netflix)가 오리지널 시리즈 <고요의 바다>의 배두나, 공유, 이준 캐스팅을 확정했다. 
19 
21 넷플릭스 VS 왓챠 일주일 사용해본 나의 선택은?
22 영화 Mute 뮤트 * 블레이드 러너 독일 버전?(넷플릭스 SF 범죄 스릴러)
23 넷플릭스 <기묘한 이야기> 시즌4 제작 재개, 9월 28일 촬영 시작
24 [도시괴담] 열대야를 날려버릴 등골 오싹한 8개의 무서운 이야기, 넷플릭스 공포영화 추천 
25 
26 한스 짐머, 넷플릭스 

## 내용 텍스트 분석

In [ ]:
!pip install scipy

In [ ]:
! pip install sklearn

In [ ]:
! pip install matplotlib

In [ ]:
! pip install seaborn

In [ ]:
! pip install Wordcloud

In [ ]:
! pip install graphviz

In [ ]:
! pip install Konlpy

In [72]:
# !pip install graphviz
# !pip install konlpy

import scipy as sp
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 그래프
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
from sklearn.tree import export_graphviz

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
                                        # 맥 os:  
rc('font', family=font_name)

In [73]:
from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *
import pickle

In [74]:
df_r = pd.read_excel("netflix_20200601_20200731.xlsx")
df_r

,Unnamed: 0,title,nickname,datetime,content
0,1,NaN,뀨우,2020. 8. 30. 12:39,이미 <내사모남 2>로 리플레이를 작성한지가 꽤 지났죠? 그러하기에 <키싱 부스> ...
1,2,넷플릭스 365일 정상을 차지하다,마시우,2020. 8. 31. 11:12,넷플릭스 무비 차트\n▣ 35주차 1위 : 365일\n집에서 볼만한 영화는 뭐가 있...
2,3,넷플릭스 호러 스릴러 영화 추천 <데블>: 우연도 자비도 없는~,젤소미나,2020. 9. 7. 23:12,"넷플릭스 추천 영화 <데블>\nDevil, 2010 밤낮이 바뀌니 새벽 시간에 멀뚱..."
3,4,메시아 - 넷플릭스,핑크팬더\n(ljb1202),2020. 9. 5. 9:00,무척이나 논쟁이 될 수밖에 없는 작품이 <메시아>다.\n어떻게 보면 이 드라마가 넷...
4,5,[넷플릭스] 웹툰 작가 최규석 & 연상호 지옥(,노루한마리\n(ravemst),16시간 전,#넷플릭스기대작 #연상호감독 #최규석 #웹툰송곳 #웹툰지옥 #넷플릭스지옥\n넷플릭스...
...,...,...,...,...,...
76,90,보건교사 안은영 젤리 몬스터와 싸우는 선생님 이야기 넷플릭스,괴수의왕,2020. 9. 2. 16:27,9월 28일에 넷플릭스 에서 방송 예정인 판타지 SF 드라마 보건교사 안은영이 넷플...
77,91,보건교사 안은영 넷플릭스 드라마 택시,괴수의왕,2020. 9. 11. 15:39,어제 저녁에 보건교사 안은영 포스터가 붙은 버스를 봤습니다.\n정유미 와 남주혁 주...
78,92,쥬라기월드 백악기 어드벤처 넷플릭스 버전 공룡 피규어가 나왔습니다.,괴수의왕,2020. 9. 2. 22:14,넷플릭스에서 9월 18일에 방송하는 쥬라기월드 백악기 어드벤처 의 공룡 피규어들이 ...
79,93,#살아있다,사과씨,2020. 8. 26. 10:40,"'조일형'감독이 연출과 각색을 맡고, '유아인', '박신혜'가 출연하는 영화 <살아..."


In [75]:
content_text = df_r['content']